In [1]:
import re
import heapq 
import copy
import sys
import math

sys.setrecursionlimit(100000)

In [2]:
threeDimMap = {
    (0,0) : (1,1,2),
    (30,180) : (1, 0.67, 1.67),
    (60,180) : (1, 0.33, 1.33),
    (90,180) : (1, 0, 1),
    (120,180) : (1, 0.33, 0.67),
    (150,180) : (1, 0.67, 0.33),
    (180,180) : (1, 1, 0),
    (150,0) : (1, 1.33, 0.33),
    (120,0) : (1, 1.67, 0.67),
    (90,0) : (1, 2, 1),
    (60,0) : (1, 1.67, 1.33),
    (30,0) : (1, 1.33, 1.67),
    
    (30,270) : (1.33, 1, 1.67),
    (60,270) : (1.67, 1, 1.33),
    (90,270) : (2, 1, 1),
    (120,270) : (1.67, 1, 0.67),
    (150,270) : (1.33, 1, 0.33),
    (150,90) : (0.67, 1, 0.33),
    (120,90) : (0.33, 1, 0.67),
    (90,90) : (0, 1, 1),
    (60,90) : (0.33, 1, 1.33),
    (30,90) : (0.67, 1, 1.67),
    
    (90,330) : (1.33, 1.67, 1),
    (90,300) : (1.67, 1.33, 1),
    (90,240) : (1.67, 0.67, 1),
    (90,210) : (1.33, 0.33, 1),
    (90,150) : (0.67, 0.33, 1),
    (90,120) : (0.33, 0.67, 1),
    (90,60) : (0.33, 1.33, 1),
    (90,30) : (0.67, 1.67, 1)
    
}

axis0_180Base=[(0, 0),(30, 0),(60, 0),(90, 0),(120, 0),(150, 0),(180, 180),(150, 180),(120, 180),(90, 180),(60, 180),(30, 180)]

axis90_270Base=[(0, 0),(30, 90),(60, 90),(90, 90),(120, 90),(150, 90),(180, 180),(150, 270),(120, 270),(90, 270),(60, 270),(30, 270)]

equatorBase = [(90,0),(90,30),(90,60),(90,90),(90,120),(90,150),(90,180),(90,210),(90,240),(90,270),(90,300),(90,330)]

def heuristic(X, Y):
    if (X in axis0_180Base) & (Y in axis0_180Base):
        indexX = axis0_180Base.index(X)
        indexY = axis0_180Base.index(Y)
        steps = abs(indexX - indexY)
        if (steps> 6):
            k = steps - 6
            steps = 6 - k
        
        return steps
    
    elif (X in axis90_270Base) & (Y in axis90_270Base):
        indexX = axis90_270Base.index(X)
        indexY = axis90_270Base.index(Y)
        steps = abs(indexX - indexY)
        if (steps> 6):
            k = steps - 6
            steps = 6 - k
        
        return steps
    
    elif (X in equatorBase) & (Y in equatorBase):
        indexX = equatorBase.index(X)
        indexY = equatorBase.index(Y)
        steps = abs(indexX - indexY)
        if (steps> 6):
            k = steps - 6
            steps = 6 - k
        
        return steps
    else:
        x = threeDimMap[X]
        y = threeDimMap[Y]
        distance = 2*math.sqrt(sum([(a - b) ** 2 for a, b in zip(x, y)]))
        return distance

    

In [3]:
heuristic((150,90), (60,180))

2.4962371682193982

In [4]:
axis0_180 = {}
axis90_270 = {}
equator = {}

def populateAxes(tileNums):
    actualPos = (int(tileNums[0]), int(tileNums[1]))
    truePos = (int(tileNums[2]), int(tileNums[3]))
                
    #populate equator
    if (actualPos[0] == 90):
        equator[actualPos] = truePos
        
    #populate axis0-180
    if (actualPos[1] == 0):
        axis0_180[actualPos] = truePos
    if (actualPos[1] == 180):
        axis0_180[actualPos] = truePos
                
    #populate axis90-270
    if(actualPos[1] == 90):
        axis90_270[actualPos] = truePos
    if(actualPos[1] == 270):
        axis90_270[actualPos] = truePos
    if(actualPos[0] == 0 & actualPos[1] == 0):
        axis90_270[actualPos] = truePos
    if(actualPos[0] == 180 & actualPos[1] == 180):
        axis90_270[actualPos] = truePos
    
def readState(filename):
    file1 = open(filename,"r")
    lines = file1.readlines()
    #print (lines[1:31])
    #print("\n")
        
    for line in lines[1:31]:
        tileNums = re.findall(r'\d+', line)
        #print(tileNums)
        if(len(tileNums)>4):
            #print(tileNums[2:6])
            populateAxes(tileNums[2:6])
        else:
            #print(tileNums)
            populateAxes(tileNums)

readState("PathN-10.mb")

In [5]:
class GlobeState:
    
    def __init__(self, axis0_180, axis90_270, equator, g, f):
        self.axis0_180 = axis0_180
        self.axis90_270 = axis90_270
        self.equator = equator
        self.g = g
        self.f = f
        
    #rotate clockwise axis 0-180
    def RC_axis0_180(self):
        newD = {}
        
        newD[(30,180)] = self.axis0_180[(0,0)]
        
        for i in [60, 90, 120, 150, 180]:
            newD[(i, 180)] = self.axis0_180[(i-30,180)]
        
        newD[(150,0)] = self.axis0_180[(180,180)]
        
        for i in [120, 90, 60, 30, 0]:
            newD[(i, 0)] = self.axis0_180[(i+30,0)]
        
        #swaps in axis90_270    
        self.axis90_270[(0,0)] = self.axis0_180[(30,0)]
        self.axis90_270[(180,180)] = self.axis0_180[150,180]
        
        #swaps in equator
        self.equator[(90,0)] = self.axis0_180[120,0]
        self.equator[(90,180)] = self.axis0_180[60,180]
        
        #new axis0_180
        self.axis0_180 = newD
    
    def RCC_axis0_180(self):
        newD = {}
        
        newD[(0,0)] = self.axis0_180[(30,180)]
        
        for i in [30, 60, 90, 120, 150]:
            newD[(i,0)] = self.axis0_180[(i-30,0)]
        
        newD[(180, 180)] = self.axis0_180[(150,0)]
        
        for i in [150, 120, 90, 60, 30]:
            newD[(i,180)] = self.axis0_180[(i+30, 180)]
            
        #swaps in axis90_270
        self.axis90_270[(0,0)] = self.axis0_180[(30,180)]
        self.axis90_270[(180, 180)] = self.axis0_180[(150, 0)]
        
        #swaps in equator
        self.equator[(90,0)] = self.axis0_180[(60,0)]
        self.equator[(90,180)] = self.axis0_180[(120, 180)]
        
        #new axis0_180
        self.axis0_180 = newD
    
    def RC_axis90_270(self):
        newD = {}
        
        newD[(30,270)] = self.axis90_270[(0,0)]
        
        for i in [60, 90, 120, 150]:
            newD[(i,270)] = self.axis90_270[(i-30, 270)]
        
        newD[(180,180)] = self.axis90_270[(150,270)]
        newD[(150, 90)] = self.axis90_270[(180,180)]
        
        for i in [120, 90, 60, 30]:
            newD[(i, 90)] = self.axis90_270[(i+30,90)]
        
        newD[(0,0)] = self.axis90_270[(30,90)]
        
        #swaps in axis0_180
        self.axis0_180[(0,0)] = self.axis90_270[(30,90)]
        self.axis0_180[(180,180)] = self.axis90_270[(150,270)]
        
        #swaps in equator
        self.equator[(90,90)] = self.axis90_270[(120,90)]
        self.equator[(90,270)] = self.axis90_270[(60,270)]
        
        #new axis90_270
        self.axis90_270 = newD
    
    def RCC_axis90_270(self):
        newD = {}
        
        newD[(0,0)] = self.axis90_270[(30, 270)]
        newD[(30,90)] = self.axis90_270[(0,0)]
        
        for i in [60, 90, 120, 150]:
            newD[(i,90)] = self.axis90_270[(i-30,90)]
            
        newD[(180,180)] = self.axis90_270[(150,90)]
        newD[(150, 270)] = self.axis90_270[(180,180)]
        
        for i in [120, 90, 60, 30]:
            newD[(i,270)] = self.axis90_270[(i+30, 270)]
        
        #sawps in axis0_180
        self.axis0_180[(0,0)] = self.axis90_270[(30, 270)]
        self.axis0_180[(180,180)] = self.axis90_270[(150, 90)]
        
        #swaps in equator
        self.equator[(90,90)] = self.axis90_270[(60,90)]
        self.equator[(90,270)] = self.axis90_270[(120,270)]
        
        #new axis90_270
        self.axis90_270 = newD
    
    def RC_equator(self):
        newD = {}
        
        newD[(90,330)] = self.equator[(90,0)]
        
        for i in [300, 270, 240, 210, 180, 150, 120, 90, 60, 30, 0]:
            newD[(90, i)] = self.equator[(90, i+30)]
        
        #swaps in axis0_180
        self.axis0_180[(90,0)] = self.equator[(90, 30)]
        self.axis0_180[(90,180)] = self.equator[(90, 210)]
        
        #swaps in axis90_270
        self.axis90_270[(90, 90)] = self.equator[(90, 120)]
        self.axis90_270[(90, 270)] = self.equator[(90, 300)]
        
        #new equator
        self.equator = newD
        
    def RCC_equator(self):
        newD = {}
        
        newD[(90,0)] = self.equator[(90,330)]
        
        for i in [30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330]:
            newD[(90, i)] = self.equator[(90, i-30)]
        
        #swaps in axis0_180
        self.axis0_180[(90,0)] = self.equator[(90, 330)]
        self.axis0_180[(90,180)] = self.equator[(90, 150)]
        
        #swaps in axis90_270
        self.axis90_270[(90, 90)] = self.equator[(90, 60)]
        self.axis90_270[(90, 270)] = self.equator[(90, 240)]
        
        #new equator
        self.equator = newD
        
    def getCurrentHeuristic(self):
        H = 0
        
        for key in self.axis0_180.keys():
            H = H + heuristic(key, self.axis0_180[key])
        
        for key in self.axis90_270.keys() - self.axis0_180.keys():
            H = H + heuristic(key, self.axis90_270[key])
        
        for key in self.equator.keys() - self.axis90_270.keys() - self.axis0_180.keys():
            H = H + heuristic(key, self.equator[key])
        
        return H/12
    
    
    def isSolved(self):
        flag = True
        for key in self.axis0_180.keys():
            if (key != self.axis0_180[key]):
                flag = False
        for key in self.axis90_270.keys():
            if (key != self.axis90_270[key]):
                flag = False
        for key in self.equator.keys():
            if (key != self.equator[key]):
                flag = False
        
        return flag
    
    

In [22]:
gs = GlobeState(axis0_180, axis90_270, equator, 0, 10000000)
#gs.RCC_axis0_180()
gs1 = copy.deepcopy(gs)
gs1.isSolved()

False

In [23]:
gs.getCurrentHeuristic()

5.791082064628188

In [24]:
gs.RCC_axis0_180()
gs.getCurrentHeuristic()

6.52707216078102

In [25]:
gs.RCC_axis0_180()
gs.getCurrentHeuristic()

6.047468205922132

In [58]:
gs1.equator

{(90, 0): (30, 0),
 (90, 120): (30, 90),
 (90, 30): (60, 0),
 (90, 90): (60, 90),
 (90, 210): (120, 180),
 (90, 270): (120, 270),
 (90, 180): (150, 180),
 (90, 300): (150, 270),
 (90, 60): (90, 60),
 (90, 150): (90, 150),
 (90, 240): (90, 240),
 (90, 330): (90, 330)}

In [59]:
def copyState(state):
    newState = GlobeState(state.axis0_180, state.axis90_270, state.equator, state.g)
    return newState

In [20]:
def Astar(n):
    #initialize open and closed lists
    openList=set()
    closedList=set()
    
    #k=0
    #add start state to openList
    openList.add(gs)
    
    while(len(openList)!=0):
        #print(closedList)
        #k=k+1
        #find state with min f val
        minVal=10000000000
        for state in openList:
            if(state.getCurrentHeuristic() + state.g < minVal):
                minVal = state.getCurrentHeuristic() + state.g
                print (minVal)
                q = state
        
        #create successors
        gs1 = copy.deepcopy(q)
        gs2 = copy.deepcopy(q)
        gs3 = copy.deepcopy(q)
        gs4 = copy.deepcopy(q)
        gs5 = copy.deepcopy(q)
        gs6 = copy.deepcopy(q)
        
        gs1.RC_axis0_180()
        gs2.RCC_axis0_180()
        gs3.RC_axis90_270()
        gs4.RCC_axis90_270()
        gs5.RC_equator()
        gs6.RCC_equator()
        
        #remove q from openList
        openList.remove(q)
        
        print(len(openList))
            
        closedList.add(q)
        
        #for each succesor, if successor in either list and f val in listed item is greater, skip this successor.
        #else add it to openList
        for state in [gs1, gs2, gs3, gs4, gs5, gs6]:
            if(state.isSolved()):
                return state
            
            flag = True
            for stateX in closedList:
                if (stateX.axis0_180 == state.axis0_180) & (stateX.axis90_270 == state.axis90_270) & (stateX.equator == state.equator):
                    #if stateX.getCurrentHeuristic() + stateX.g < f:
                    flag = False
            
            if(flag==False):
                continue
            
            tentativeG = q.g + 1
               
            if (tentativeG < state.g):
                state.g = tentativeG
                state.f = state.g + state.getCurrentHeuristic()
            
            
            for stateX in openList:
                
                if (stateX.axis0_180 == state.axis0_180) & (stateX.axis90_270 == state.axis90_270) & (stateX.equator == state.equator):
                    #if stateX.getCurrentHeuristic() + stateX.g < f:
                    flag = False
            
            if(flag):
                openList.add(state)
        
        
        

        

In [ ]:
gh = Astar(8)

5.791082064628188
0
5.17559795685256
5
5.7558415226515285
4.5465217640555915
4.48476764170341
9
5.7558415226515285
3.8556914489064433
13
5.7558415226515285
3.4443659660777874
17
5.7558415226515285
4.383104268934585
3.2436867533832134
21
5.7558415226515285
4.383104268934585
3.0481423930261538
3.004162789558739
25
5.7558415226515285
3.921456594855505
3.593258113859688
3.0481423930261538
3.02997462492101
29
5.7558415226515285
3.921456594855505
3.6544083905894422
3.1677635978972845
3.0481423930261538
33
5.7558415226515285
3.3961843869161323
2.979422192963847
37
5.7558415226515285
3.3961843869161323
3.2640618776218555
3.0481423930261538
41
5.7558415226515285
3.3961843869161323
3.2640618776218555
3.2406208950550925
3.1677635978972845
3.141951762535013
2.979422192963847
45
5.7558415226515285
3.3961843869161323
3.2640618776218555
3.2406208950550925
3.1677635978972845
3.141951762535013
2.6460888596305137
49
5.7558415226515285
3.3961843869161323
3.2640618776218555
3.2406208950550925
3.0862920361

371
3.140422395948517
3.038293495480728
3.0236123928333107
2.992482666640703
2.8922841675341284
2.8123891745868197
375
3.140422395948517
3.038293495480728
3.0236123928333107
2.992482666640703
2.8922841675341284
379
3.140422395948517
3.038293495480728
3.0236123928333107
2.992482666640703
2.759630046895369
383
3.140422395948517
3.038293495480728
3.0236123928333107
2.5506588611510934
387
3.140422395948517
3.038293495480728
3.0236123928333107
2.7729852542032103
391
3.140422395948517
3.038293495480728
3.0236123928333107
2.992482666640703
2.972107542402061
2.9368407191139525
2.9056393748419693
395
3.140422395948517
3.038293495480728
3.0236123928333107
2.992482666640703
2.972107542402061
2.9368407191139525
399
3.140422395948517
3.038293495480728
3.0236123928333107
2.992482666640703
2.939723538949252
403
3.140422395948517
3.038293495480728
3.0236123928333107
2.992482666640703
2.7307523532049767
407
3.140422395948517
3.038293495480728
3.0236123928333107
2.992482666640703
2.972107542402061
2.954

726
3.336642220007377
3.140422395948517
3.1035073857806186
3.0640856865383093
2.943229848711668
2.9409061981300777
730
3.336642220007377
3.140422395948517
3.1035073857806186
3.0640856865383093
2.943229848711668
2.9409061981300777
734
3.336642220007377
3.140422395948517
3.1035073857806186
3.0640856865383093
2.943229848711668
2.7876663568506275
738
3.336642220007377
3.140422395948517
3.1035073857806186
3.0640856865383093
2.943229848711668
2.894536200036343
742
3.336642220007377
3.140422395948517
3.1035073857806186
3.0640856865383093
2.943229848711668
746
3.336642220007377
3.140422395948517
3.1035073857806186
3.0640856865383093
3.038293495480728
2.976539373128547
2.9597500903362337
2.9543330235172944
2.952080991015079
2.947943893919579
2.7876663568506275
750
3.336642220007377
3.140422395948517
3.1035073857806186
3.0640856865383093
3.038293495480728
2.976539373128547
2.9597500903362337
2.9543330235172944
2.952080991015079
2.947943893919579
2.71444270798246
754
3.336642220007377
3.140422395

2.8639834219561444
2.8477453948130034
2.790549176685928
2.692181902951963
2.6902790594999773
894
3.336642220007377
3.140422395948517
3.1035073857806186
3.0640856865383093
3.038293495480728
2.976539373128547
2.9597500903362337
2.9543330235172944
2.952080991015079
2.9163761979372347
2.8639834219561444
2.8477453948130034
2.790549176685928
2.692181902951963
898
3.336642220007377
3.140422395948517
3.1035073857806186
3.0640856865383093
3.038293495480728
2.976539373128547
2.9597500903362337
2.9543330235172944
2.952080991015079
2.9163761979372347
2.81654405054102
2.790549176685928
2.7612506758659308
2.727869533369676
2.727166511758647
902
3.336642220007377
3.140422395948517
3.1035073857806186
3.0640856865383093
3.038293495480728
2.976539373128547
2.9597500903362337
2.9543330235172944
2.952080991015079
2.9163761979372347
2.81654405054102
2.790549176685928
2.7612506758659308
2.727869533369676
906
3.336642220007377
3.140422395948517
3.1035073857806186
3.0640856865383093
3.038293495480728
2.976539

1047
3.336642220007377
3.140422395948517
3.1035073857806186
3.025515236285296
2.976539373128547
2.9749903622373606
2.81850134941225
2.81654405054102
2.6721553515649816
2.2189461567883213
1051
3.336642220007377
3.140422395948517
3.1035073857806186
3.025515236285296
2.976539373128547
2.9749903622373606
2.81850134941225
2.81654405054102
2.6721553515649816
2.107572864796744
1055
3.336642220007377
3.140422395948517
3.1035073857806186
3.025515236285296
2.976539373128547
2.9749903622373606
2.81850134941225
2.774239531463411
2.6721553515649816
2.31654405054102
1059
3.336642220007377
3.140422395948517
3.1035073857806186
3.025515236285296
2.976539373128547
2.9749903622373606
2.81850134941225
2.774239531463411
2.6721553515649816
2.6047616630408195
2.4279173425325973
1063
3.336642220007377
3.140422395948517
3.1035073857806186
3.025515236285296
2.976539373128547
2.9749903622373606
2.81850134941225
2.774239531463411
2.6721553515649816
2.6047616630408195
2.472107542402061
1067
3.336642220007377
3.140

...

2.049396670286354
5944
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.204960162147395
2.2026365115658044
2.180430161954552
2.1673696882776956
5947
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.204960162147395
2.2026365115658044
2.180430161954552
2.1673696882776956
5951
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.204960162147395
2.2026365115658044
2.180430161954552
2.1673696882776956
5955
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.204960162147395
2.2026365115658044
2.180430161954552
2.178472863083322
2.173830435954787
2.1673696882776956
5958
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.204960162147395
2.2026365115658044
2.180430161954552
2.1784728630833

6060
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.204960162147395
2.2026365115658044
2.180430161954552
2.178472863083322
2.173830435954787
2.1673696882776956
2.1473431368907145
2.145163338666443
6064
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.204960162147395
2.2026365115658044
2.180430161954552
2.178472863083322
2.173830435954787
2.1673696882776956
2.1473431368907145
1.8118300053331098
6068
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.204960162147395
2.2026365115658044
2.180430161954552
2.178472863083322
2.173830435954787
2.1673696882776956
2.1473431368907145
2.038293495480728
1.9695732954184215
6072
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.204960162147395
2.2026365115658044
2.180430161954552
2.178472863083322
2.

2.167369688277696
2.1673696882776956
6178
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.204960162147395
2.2026365115658044
2.180430161954552
2.178472863083322
2.173830435954787
2.1693269871489256
2.167369688277696
2.1673696882776956
1.7049601621473947
6182
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.204960162147395
2.2026365115658044
2.180430161954552
2.178472863083322
2.173830435954787
2.1693269871489256
2.167369688277696
2.1673696882776956
1.8404971026214536
6186
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.204960162147395
2.2026365115658044
2.180430161954552
2.178472863083322
2.173830435954787
2.1693269871489256
2.167369688277696
2.1673696882776956
2.013763495287885
6189
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.

6300
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.204960162147395
2.2026365115658044
2.180430161954552
2.173830435954787
2.1693269871489256
2.167369688277696
2.1673696882776956
6303
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.204960162147395
2.2026365115658044
2.180430161954552
2.173830435954787
1.722734681032161
6307
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.204960162147395
2.2026365115658044
2.180430161954552
2.173830435954787
1.7449410306434137
6311
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.204960162147395
2.2026365115658044
2.180430161954552
2.173830435954787
1.8206095295571458
6315
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.20496016214739

2.1317365132792268
1.949198171179779
6432
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.204960162147395
2.2026365115658044
2.180430161954552
2.173830435954787
2.1317365132792268
1.900504522504454
6436
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.204960162147395
2.2026365115658044
2.180430161954552
2.173830435954787
2.1317365132792268
6440
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.204960162147395
2.2026365115658044
2.180430161954552
2.173830435954787
2.1582238123432993
2.1317365132792268
1.9761730214181863
6444
2.980676470224047
2.862703452232706
2.8340363549443626
2.458470120612795
2.3738066270383325
2.3340363549443626
2.204960162147395
2.2026365115658044
2.180430161954552
2.173830435954787
2.1582238123432993
2.1317365132792268
1.900504522504454
6448
2.980676470224047
2.8627034522327

In [16]:
gh.axis0_180

{(0, 0): (0, 0),
 (30, 0): (30, 0),
 (60, 0): (60, 0),
 (90, 0): (90, 0),
 (120, 0): (120, 0),
 (150, 0): (150, 0),
 (180, 180): (180, 180),
 (150, 180): (150, 180),
 (120, 180): (120, 180),
 (90, 180): (90, 180),
 (60, 180): (60, 180),
 (30, 180): (30, 180)}

In [2]:
int("inf")

ValueError: invalid literal for int() with base 10: 'inf'

In [7]:
def Astar(gs):
    k=0;
    h = []
    heapq.heappush(h, (gs.getCurrentHeuristic(), gs))
    
    while(h[0][1].isSolved() == False):    
        gs = heapq.heappop(h)[1]
        k=k+1
        print(k)
        gs1 = copy.copy(gs)
        gs2 = copy.copy(gs)
        gs3 = copy.copy(gs)
        gs4 = copy.copy(gs)
        gs5 = copy.copy(gs)
        gs6 = copy.copy(gs)
        
        gs1.RC_axis0_180()
        gs2.RCC_axis0_180()
        gs3.RC_axis90_270()
        gs4.RCC_axis90_270()
        gs5.RC_equator()
        gs6.RCC_equator()
             
        a = gs1.getCurrentHeuristic()+k
        b = gs2.getCurrentHeuristic()+k
        c = gs3.getCurrentHeuristic()+k
        d = gs4.getCurrentHeuristic()+k
        e = gs5.getCurrentHeuristic()+k
        f = gs6.getCurrentHeuristic()+k
        
        if ((a,gs1) not in h):
            heapq.heappush(h, (a, gs1))
        if ((b,gs2) not in h):
            heapq.heappush(h, (b, gs2))
        if ((c,gs3) not in h):
            heapq.heappush(h, (c, gs3))
        if ((d,gs4) not in h):
            heapq.heappush(h, (d, gs4))
        if ((e,gs5) not in h):
            heapq.heappush(h, (e, gs5))
        if ((f,gs6) not in h):
            heapq.heappush(h, (f, gs6))
        #print(type(d))
    return k

In [ ]:

    
threeDimMap = {
    (0,0) : (1,1,2),
    (30,180) : (1, 0.67, 1.67),
    (60,180) : (1, 0.33, 1.33),
    (90,180) : (1, 0, 1),
    (120,180) : (1, 0.33, 0.67),
    (150,180) : (1, 0.67, 0.33),
    (180,180) : (1, 1, 0),
    (150,0) : (1, 1.33, 0.33),
    (120,0) : (1, 1.67, 0.67),
    (90,0) : (1, 2, 1),
    (60,0) : (1, 1.67, 1.33),
    (30,0) : (1, 1.33, 1.67),
    
    (30,270) : (1.33, 1, 1.67),
    (60,270) : (1.67, 1, 1.33),
    (90,270) : (2, 1, 1),
    (120,270) : (1.67, 1, 0.67),
    (150,270) : (1.33, 1, 0.33),
    (150,90) : (0.67, 1, 0.33),
    (120,90) : (0.33, 1, 0.67),
    (90,90) : (0, 1, 1),
    (60,90) : (0.33, 1, 1.33),
    (30,90) : (0.67, 1, 1.67),
    
    (90,330) : (1.33, 1.67, 1),
    (90,300) : (1.67, 1.33, 1),
    (90,240) : (1.67, 0.67, 1),
    (90,210) : (1.33, 0.33, 1),
    (90,150) : (0.67, 0.33, 1),
    (90,120) : (0.33, 0.67, 1),
    (90,60) : (0.33, 1.33, 1),
    (90,30) : (0.67, 1.67, 1)
    
}

def heuristic(X, Y):
    x = threeDimMap[X]
    y = threeDimMap[Y]
    distance = 2*math.sqrt(sum([(a - b) ** 2 for a, b in zip(x, y)]))
    return distance



def BFS():
    openList = []
    openList.append(gs)
    
    while (len(openList) > 0):
        q = openList.pop()
        
        if(q.isSolved()):
            return q
        
        gs1 = copy.deepcopy(q)
        gs2 = copy.deepcopy(q)
        gs3 = copy.deepcopy(q)
        gs4 = copy.deepcopy(q)
        gs5 = copy.deepcopy(q)
        gs6 = copy.deepcopy(q)
        
        gs1.RC_axis0_180()
        gs2.RCC_axis0_180()
        gs3.RC_axis90_270()
        gs4.RCC_axis90_270()
        gs5.RC_equator()
        gs6.RCC_equator()
        
        for state in (gs1, gs2, gs3, gs4, gs5, gs6):
            flag = True
            
            for stateX in openList:
                if (stateX.axis0_180 == state.axis0_180) & (stateX.axis90_270 == state.axis90_270) & (stateX.equator == state.equator):
                    flag = False
                
            if(flag):
                openList.append(state)
                
        print (openList)